### 正则化

#### "Regularization"在英语中的基本含义是"使规则化"、"使正常化"或"标准化"的过程。在机器学习语境中，它通常译为"正则化"。
#### Regularization 的核心思想是 “约束” 或 “惩罚”。它是一种用于 防止模型过度拟合 (Overfitting) 的技术。
为了防止overfitting的技术，通过惩罚模型的复杂度来约束模型的学习能力  
它会给模型增加一些“成本”，使得模型在追求低训练误差的同时，也会权衡模型本身的复杂度。
- L1 正则化(Lasso Regression回归)
    - 惩罚项为权重的绝对值之和（λ∑|w|）。
    稀疏性 (Sparsity)：L1 正则化倾向于将一些模型参数（wi）推向零。这意味着 L1 正则化可以自动进行特征选择，将不重要的特征的权重置零，从而得到一个更稀疏的模型。
    - L1 范数: 正则化项是模型权重绝对值之和（L1 范数）。
- L2 正则化（Ridge Regression 岭回归）
    - 正则化项: $ \lambda\sum{w^2}$
    - 权重衰减 (Weight Decay)：L2 正则化倾向于将模型参数推向零，但通常不会将它们完全置零。它使得模型的权重更加平滑，减小了对单个特征的依赖。
    - L2 范数: 正则化项是模型权重平方之和（L2 范数）。
- Elastic Net 正则化 $ \lambda\sum{\left| w_i\right|} + \lambda\sum{w^2}$
    - 结合了L1 和L2的优点
- 其他形式的正则化（针对特定模型或情况）
    - Dropout (在神经网络中)：在训练过程中，随机“丢弃”一部分神经元及其连接，迫使网络学习冗余的表示，提高鲁棒性。
    - Batch Normalization (在神经网络中)：对每一层神经网络的输入进行归一化，可以加速训练，并且在一定程度上起到正则化作用。

一些问题：  
1.为什么L2正则化要叫岭回归：
L2 正则化通过给模型系数的平方和增加惩罚项，使得在训练过程中，即使面对多重共线性，模型也不会赋予某些系数过大的值。这样做有效地**“压低”**了系数的大小，增加了估计的稳定性，这种“压低”的过程在可视化时（岭图）呈现出类似山脊的形状，因此得名“岭回归”  
2.强制学习冗余表示 (Redundancy)
减少神经元之间的协同适应： 在没有 Dropout 的情况下，神经元可能会形成高度协同的“共适应”关系，即某些神经元组合在一起才能正确地进行预测。  
3. Dropout 强行打断了这种固定的共适应关系。
每个神经元更独立地学习特征： 由于不知道哪些神经元会在下一次训练中被丢弃，每个神经元都需要学习对输入数据更具鲁棒性、更独立的特征。即使它所在的“子网络”中缺少某些其他神经元，它仍然能够提供有用的信息。

在神经网络中 L2正则会为什么要叫做权重衰减：  
我们通过反向传播的过程来看（利用梯度更新W）
原始的梯度下降更新是：  

$$ w_{new} = w_{old} - \alpha\dfrac{\sigma  J(\theta)}{\sigma w}$$

加入L2正则化后的更新：  
$$ w_{new} = w_{old} - \alpha ( \dfrac{\sigma  J(\theta)}{\sigma w} + \lambda w_{old})$$

展开之后：  
$$ w_{new} = w_{old}(1- \alpha\lambda) - \alpha\dfrac{\sigma  J(\theta)}{\sigma w}$$ 

衰减项的贡献： 
(1−αλ) 这个项会使得权重在每次更新时都乘以一个小于 1 的系数。这就像是在每次迭代中，“衰减”了权重的数值。

我认为的dropout 和L1正则化的一些相同点和区别：
- dropout是每次随机丢弃每一层的一部分参数，或者说是选择网络中一个子网络进行训练
- L1正则化则是加入惩罚 使参数趋向于0
- 区别是正则化直接作用于参数 而dropout是间接作用于参数 

### nn.Module  
核心有两大块：
- 定义网络中有哪些元素， def __init__() ：模块会自动将器内部的nn.Moudle的实例内容r(nn.Linear、nn.Conv2d)如捕捉，进行统一的管理  
nn.Module 实现了一个树形结构，每个模块可以包含多个子模块，形成层次结构：
_modules: 字典，存储所有已注册的子模块  
_parameters: 字典，存储所有已注册的参数  
_buffers: 字典，存储所有已注册但不需要梯度的张量（如 BatchNorm 的 running_mean）  
**内部工作原理**
当你给 self 分配一个 nn.Module 子类的实例时，以下是幕后发生的事情：
当你执行 self.conv1 = nn.Conv2d(3, 16, 3) 时，nn.Module 的 __setattr__ 方法被调用
__setattr__ 检查你分配的对象是否为 nn.Module 的实例
如果是，它将该模块添加到 self._modules 字典中，键为属性名（如 'conv1'）
这使 PyTorch 能够：
跟踪所有子模块
递归地应用操作（如 .to(device)、.train()）
收集所有参数进行优化
- 定义网络中这些元素的连接方式，重写 def forward(self,x): pytorch会自动向前传播，  
Module(x) 会自动调用moudle.forward(slef,x)
- 其他的就是nn.Moudle 原本的功能
    - Module.train() 将网络设置为训练模式
    - Module.eval() 将模型设置为评估函数，不启动Dropout(保证输出的准确) 不会构建计算图，节省计算资源
    - Module.parameters()
    - Module.named_parameters() 返回名字和参数
    - Module.to(device) 会递归的将所有参数和子模块移动到指定设备 注意返回的是副本，需要赋值给原来的module



nn.Module 的属性注册工作流程
当你创建一个 nn.Module 子类实例时，其 __init__ 方法被调用
在 __init__ 中，你给 self 分配各种属性
当分配属性时，nn.Module 的 __setattr__ 方法会：
检查值是否是 Parameter，如是则添加到 _parameters
检查值是否是 Module，如是则添加到 _modules
检查值是否是 Tensor 且 persistent=True，如是则添加到 _buffers
否则，正常设置属性
这种机制确保了你的模型结构被正确跟踪，所有参数都可以被优化器访问，并且操作（如移动到 GPU）可以应用到整个模型。

这就是为什么将层"作为属性定义"如此重要 - 这是 PyTorch 自动构建计算图和管理参数的关键机制。

In [7]:
import torch
import torch.nn as nn

class mymodule(nn.Module):
    def __init__(self,num_inputs,hidden_size,num_outputs):
        super(mymodule,self).__init__()
        """
            super() 是Python内置函数，用于获取父类或兄弟类的临时代理对象，使你能够调用它们的方法。
            super(type, obj) type: 一个类（通常是当前类） obj: 类的实例（通常是self）
            其实也可以写成super(nn.Module,self) 效果是相同的，但是不符合python的惯例
            python3 可以简写为 super().__init__()
            注意 python不会自动调用父类的初始化函数，这与C++不同
            Python 的核心设计原则之一是“显式优于隐式”
        """

        self.fc1 = nn.Linear(num_inputs,hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size,num_outputs)

    
    def forward(self,x):
        out = self.f1(x)
        out = self.relu(out)
        out = self.f2(out)

        return out

        

In [8]:
net = mymodule(28*28,64,10)

In [ ]:
"""
参数实际上存储在 self.conv1 和 self.fc1 这些子模块中，而不是直接存储在根模块 MyNet 的 _parameters 中。
"""

net._parameters



OrderedDict()

In [14]:
net._modules

OrderedDict([('fc1', Linear(in_features=784, out_features=64, bias=True)),
             ('relu', ReLU()),
             ('fc2', Linear(in_features=64, out_features=10, bias=True))])

In [ ]:
# 2. 使用 named_parameters() 方法（显示名称和参数）
for name, param in net.named_parameters(): 
    # type(net.named_parameters()) 也是生成器
    print(f"{name}: {param.shape}")

fc1.weight: torch.Size([64, 784])
fc1.bias: torch.Size([64])
fc2.weight: torch.Size([10, 64])
fc2.bias: torch.Size([10])


In [ ]:
for param in net.parameters(): 
    # type(net.parameters()) = gernerator 会返回一个生成器
    print(param.shape)

torch.Size([64, 784])
torch.Size([64])
torch.Size([10, 64])
torch.Size([10])


In [26]:
type(net.named_parameters())

generator

In [31]:
next(net.parameters()).device

device(type='cpu')

In [36]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
net = net.to(device)
next(net.parameters()).device

device(type='cpu')

In [40]:

torch.cuda.is_available()

False

In [ ]:
# 嵌套模块

class ComplexModel(nn.Module):
    def __init__(self):
        super(ComplexModel, self).__init__()
        
        # 子模块
        self.feature_extractor = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )
        
        self.classifier = nn.Sequential(
            nn.Linear(16 * 14 * 14, 128),
            nn.ReLU(),
            nn.Linear(128, 10)
        )
        
    def forward(self, x):
        x = self.feature_extractor(x)
        x = x.view(x.size(0), -1)  # 展平
        x = self.classifier(x)
        return x